# Wavelet21 Method Example

This notebook demonstrates the Wavelet21 method for structural breakpoint detection using wavelet decomposition and frequency domain analysis.


In [ ]:
# Setup: Install required packages first
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "../requirements.txt"])


In [ ]:
# Setup: import from local package
import sys, pathlib

project_root = pathlib.Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from StructualBreak import compute_predictors_for_values, run_batch, Wavelet21Method
print("Imports OK.")
